# EDA: Registre d'explotacions ramaderes

In [80]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Informació del *dataset*

El *dataset* escollit és el **Registre d'explotacions ramaderes de Catalunya**, que es pot trobar a [Dades Obertes de Catalunya](https://analisi.transparenciacatalunya.cat/Medi-Rural-Pesca/Registre-d-explotacions-ramaderes/7bpt-5azk). Aquest *dataset* conté informació sobre les explotacions ramaderes de Catalunya, amb dades com la seva ubicació, el tipus d'explotació, el nombre d'animals que hi ha, etc. Dintre de cadascuna de les seccions, les explotacions es classifiquen en diferents subseccions en funció de la finalitat productiva.

Les dades les proporciona el **Departament d'Acció Climàtica, Alimentació i Agenda Rural**. Aquest s'encarrega d'actualitzar les dades anualment. L'última actualització és del 8 de novembre del 2023, per tant, les dades són ben actuals.

El joc de dades conté més de 26.000 registres i 49 variables. Cada fila és una explotació ramadera i les columnes que conté mostren informació identificativa de les explotacions i característiques d'aquestes. Les variables són de tipus numèric, de text i de data.

Totes les dades que es mostren són públiques. De totes maneres, hi ha informació sobre empreses privades i particulars, per tant, s'ha de tenir cura amb la privacitat de les dades.

A continuació carreguem les dades, en visualitzem l'estructura i mostrem les primeres files:

In [81]:
# Load dataset
df = pd.read_csv('../data/raw/Registre_d_explotacions_ramaderes.csv')

In [82]:
# Convert postal code to object
df['Codi postal explotació'] = df['Codi postal explotació'].astype('object')

# Convert date columns to datetime
df['Data canvi estat explotació'] = pd.to_datetime(df['Data canvi estat explotació'])
df['Data canvi estat subexplotació'] = pd.to_datetime(df['Data canvi estat subexplotació'])
df['Data classificació zootècnica'] = pd.to_datetime(df['Data classificació zootècnica'])
df['Data actualització capacitat'] = pd.to_datetime(df['Data actualització capacitat'])

# See the structure of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Marca oficial                   25998 non-null  object        
 1   Codi REGA                       25998 non-null  object        
 2   Estat explotació                25998 non-null  object        
 3   Data canvi estat explotació     25998 non-null  datetime64[ns]
 4   Nom explotació                  25998 non-null  object        
 5   Adreça explotació               25998 non-null  object        
 6   Codi postal explotació          25998 non-null  object        
 7   Servei territorial explotació   25998 non-null  object        
 8   Província explotació            25998 non-null  object        
 9   Comarca explotació              25998 non-null  object        
 10  Municipi explotació             25998 non-null  object        
 11  CO

In [83]:
# See the first rows of the dataset
df.head()

,Marca oficial,Codi REGA,Estat explotació,Data canvi estat explotació,Nom explotació,Adreça explotació,Codi postal explotació,Servei territorial explotació,Província explotació,Comarca explotació,...,Capacitat transició,Capacitat estants,Capacitat transhumants,Capacitat AN>6M no repr,Capacitat ous,Cap núm. total animals,Total Cap ponedores,Total URM,Total Nitrogen,Data actualització capacitat
0,0010AA,ES250010030750,Activa,1997-01-21,CASA JULIU - JORDI GASA,MASIA JULIU,25651,Alt Pirineu,Lleida,Pallars Jussà,...,0,0,0,0,0,0,0,56,7515,2021-04-10
1,0010AA,ES250010030750,Activa,1997-01-21,CASA JULIU - JORDI GASA,MASIA JULIU,25651,Alt Pirineu,Lleida,Pallars Jussà,...,0,0,0,0,0,0,0,29088,17574,2023-07-21
2,0010AA,ES250010030750,Activa,1997-01-21,CASA JULIU - JORDI GASA,MASIA JULIU,25651,Alt Pirineu,Lleida,Pallars Jussà,...,0,0,0,0,0,0,0,4,2552,2014-11-21
3,0010AA,ES250010030750,Activa,1997-01-21,CASA JULIU - JORDI GASA,MASIA JULIU,25651,Alt Pirineu,Lleida,Pallars Jussà,...,0,0,0,4,0,0,0,4,2552,2014-11-21
4,0010AB,ES250010031100,Activa,1997-04-21,CASA GONELLA,BOIXOLS,25651,Alt Pirineu,Lleida,Pallars Jussà,...,475,0,0,0,0,0,0,19702,1187525,2021-04-10


## Preparació de les dades

En aquest apartat examinarem el dataset per tal d'esbrinar si el conjunt de dades conté valors perduts, valors extrems, etc. Estudiarem si cal homogeneïtzar les dades categòriques i si caldrà extreure característiques.

### Tractament dels valors perduts

In [84]:
# Detectar els valors perduts en cada columna
missing_values = df.isnull().sum()
missing_values_percentage = (missing_values / len(df)) * 100

missing_values_percentage.sort_values(ascending=False)

Codi zootècnic                    99.857681
Forma de cria                     94.326487
Integradora                       76.748211
Capacitat productiva              74.694207
Nom ADS                           55.608124
Subespècie                        48.515270
LATITUD explotació                33.675667
LONGITUD explotació               33.675667
Data classificació zootècnica     25.898146
Classificació zootècnica          25.563505
Petita capacitat                   1.715517
Data actualització capacitat       1.384722
Criteri de sostenibilitat          1.330872
Sistema productiu                  1.303946
Autoconsum                         1.303946
COOR Y explotació                  0.011539
COOR X explotació                  0.011539
Capacitat femelles                 0.000000
Capacitat cria                     0.000000
Capacitat reposició                0.000000
Capacitat engreix                  0.000000
Capacitat recria                   0.000000
Capacitat transició             

L'anàlisi de les dades mancants del dataset "Registre d'explotacions ramaderes de Catalunya" revela els següents punts:

- Valors Perduts: Algunes columnes tenen un nombre significatiu de valors mancants. Per exemple:
    - ``Codi zootècnic`` té gairebé tots els valors mancants (99.86%).
    - ``Forma de cria`` també té una gran quantitat de valors mancants (94.33%).
    - ``Integradora`` i ``Capacitat productiva`` tenen un 76.75% i un 74.69% de valors mancants, respectivament.
    - ``Subespècie`` té un 48.52% de valors mancants.
    - ``LATITUD explotació`` i ``LONGITUD explotació`` tenen aproximadament un 33.68% de valors mancants cadascuna.

- Columnes Complertes: Diverses columnes com ``Marca oficial``, ``Codi REGA``, ``Tipus explotació``, ``Espècie``, etc., no tenen cap valor mancant.

Tant ``Codi zootècnic`` com ``Forma de cria`` tenen un nombre significatiu de valors mancants.  Això és problemàtic, però abans d'eliminar-les, estudiem si estem davant de valors perduts o si realment no hi ha informació disponible. Per exemple, si una explotació no té cap animal, no tindrà cap valor per a ``Codi zootècnic``. Per tant, no es tracta d'un valor perdut, sinó que no hi ha informació disponible. Això també passa amb ``Forma de cria``. Si una explotació no cria animals, no tindrà cap valor per aquesta columna. Per tant, no es tracta d'un valor perdut, sinó que no hi ha informació disponible.

In [85]:
df['Codi zootècnic'].unique()

array([nan, 'ES09RS11P', 'ES09RS18P', 'ES09ET02A', 'ES09RS12P',
       'ES09RS28P', 'ES09RS20P', 'ES09RS32P', 'ES09RS03E', 'ES09RS05P',
       'ES09RS22P', 'ES09RS29P', 'ES09RS19P', 'ES09RS26P', 'ES09059',
       'ES09RS09P', 'ES09RS34P', 'ES09RS35P', 'ES09RS17P', 'ES09RS06P',
       'ES09RS15P', 'ES09RS13P', 'ES09RS02P', 'ES09RS31P', 'ES09ET01E',
       'ES09RS24P', 'ES09RS16P', 'ES09RS27P', 'ES09RS03P', 'ES09RS23P',
       'ES09RS30P', 'ES09RS21P', 'ES09RS37P', 'ES09RS33P', 'ES09060'],
      dtype=object)

El ``Codi zootècnic`` és un identificador per a un centre o equip autoritzat, dedicat a la recollida, emmagatzematge o distribució de material genètic. Aquesta informació no és rellevant per a l'anàlisi que es vol fer, per tant, es pot eliminar la columna.

In [86]:
# Suprimir la columna 'Codi zootècnic'
df = df.drop(columns=['Codi zootècnic'])

In [87]:
df['Forma de cria'].unique()

array([nan, 'Galliner en interior', 'Ecològica', "A l'aire lliure",
       'A terra', 'En gàbia', 'Extensiu en interior', 'Ecològic',
       'Camper tradicional', 'Camper', 'Camper criat en total llibertat'],
      dtype=object)

La columnna ``Forma de cria`` és una classificació atorgada segons la forma de cria de les aus de corral amb orientació productiva producció d'ous de consum (producció ecològica, en camp, a terra o en gàbies) o amb orientació productiva producció per carn (etiquetatge facultatiu).

És evident, que una explotació no tindrà aquest camp si no es dedica a la cria d'aus de corral. Per tant, és normal que no hi hagi informació disponible per molts registres. De moment, no eliminarem la columna, però més endavant, si no és rellevant per a l'anàlisi, la podem eliminar.

La columna ``Integradora`` és l'empresa amb la qual l'explotació ramadera té una relació contractual. Aquesta informació no és rellevant per a l'anàlisi que es vol fer, per tant, es pot eliminar la columna.

In [88]:
# Suprimir la columna 'Integradora'
df = df.drop(columns=['Integradora'])

La columma ``Capacitat productiva`` és la classificació establerta per l'espècie porcina i la família apícola. En les explotacions porcines hi ha 4 grups (explotació de petita capacitat, grup I, grup II, grup III). En el cas de les explotacions apícoles es diferencien 2 grups d'explotacions: les professionals i les no professionals.

Aquesta informació ens pot ser útil més endavant, així que no eliminarem la columna.

La columna ``Nom ADS`` també té molts valors perduts i la seva informació només és útil a nivell burocràtic. Per tant, es pot eliminar la columna.

In [89]:
# Suprimir la columna 'Nom ADS'
df = df.drop(columns=['Nom ADS'])

La resta de variables les deixem tal i com estan.

### Indentificació de les dades categòriques

Volem veure quines columnes són categòriques. Seleccionem les variables que tenen tipus `object`. I un cop tenim la llista, veiem quines són categòriques i quines no.

In [90]:
# Seleccionar les variables categòriques
# Identificar les columnes categòriques del dataframe 'df'
categorical_columns = df.select_dtypes(include=['object']).columns
categorical_columns.tolist()

['Marca oficial',
 'Codi REGA',
 'Estat explotació',
 'Nom explotació',
 'Adreça explotació',
 'Codi postal explotació',
 'Servei territorial explotació',
 'Província explotació',
 'Comarca explotació',
 'Municipi explotació',
 'LATITUD explotació',
 'LONGITUD explotació',
 'Tipus explotació',
 'Espècie',
 'Subespècie',
 'Tipus subexplotació',
 'Estat subexplotació',
 'Classificació zootècnica',
 'Forma de cria',
 'Autoconsum',
 'Sistema productiu',
 'Criteri de sostenibilitat',
 'Petita capacitat',
 'Capacitat productiva']

D'aquesta llista de variables, totes es poden considerar categòriques excepte:
- ``LATITUD explotació`` i ``LONGITUD explotació``: Són volors numèric (coordenades), encara que per les seves característiques ens interessa més guardar-los com a text o ``object``.
- ``Marca oficial`` i ``Codi REGA``: Tot i que són text, aquestes columnes podrien actuar com a identificadors únics per a cada explotació. Depenent de l'anàlisi, podrien no ser tractades com a variables categòriques típiques, ja que podrien no aportar informació útil en models predictius o en comparacions estadístiques.
- ``Nom explotació`` i ``Adreça explotació``: Són texts que identifiquen l'explotació. Podrien actuar com a identificador únic per a cada explotació. Depenent de l'anàlisi, podria no ser tractada com a variable categòrica típica, ja que podria no aportar informació útil en models predictius o en comparacions estadístiques.

Per tant, les suprimim de la llista de variables categòriques:

In [91]:
# Eliminar les columnes especificades de la llista de columnes categòriques
categorical_columns = categorical_columns.drop([
    'LATITUD explotació',
    'LONGITUD explotació',
    'Marca oficial',
    'Codi REGA',
    'Nom explotació',
    'Adreça explotació'])
categorical_columns.tolist()

['Estat explotació',
 'Nom explotació',
 'Adreça explotació',
 'Codi postal explotació',
 'Servei territorial explotació',
 'Província explotació',
 'Comarca explotació',
 'Municipi explotació',
 'Tipus explotació',
 'Espècie',
 'Subespècie',
 'Tipus subexplotació',
 'Estat subexplotació',
 'Classificació zootècnica',
 'Forma de cria',
 'Autoconsum',
 'Sistema productiu',
 'Criteri de sostenibilitat',
 'Petita capacitat',
 'Capacitat productiva']

Canviem a tipus `category` les variables categòriques:

In [94]:
# Convertir les columnes categòriques identificades a tipus 'category'
for col in categorical_columns:
    df[col] = df[col].astype('category')

# Verificar els tipus de dades actualitzats per aquestes columnes
df.dtypes

Marca oficial                             object
Codi REGA                                 object
Estat explotació                        category
Data canvi estat explotació       datetime64[ns]
Nom explotació                          category
Adreça explotació                       category
Codi postal explotació                  category
Servei territorial explotació           category
Província explotació                    category
Comarca explotació                      category
Municipi explotació                     category
COOR X explotació                        float64
COOR Y explotació                        float64
LATITUD explotació                        object
LONGITUD explotació                       object
Tipus explotació                        category
Espècie                                 category
Subespècie                              category
Tipus subexplotació                     category
Estat subexplotació                     category
Data canvi estat sub

## Visualitzar les dades

### Distribució de les variables categòriques